In [ ]:
#hide

# RaDICaL Dataset SDK

![Build Status](https://github.com/moodoki/radical_sdk/workflows/CI/badge.svg)

> The RaDICaL Dataset: A synchronized and calibrated low-level Radar, RGB-D and IMU dataset.

This is pre-alpha research quality code. Bug reports are very much appreciated.

# TODO

 - [ ] Read from aligned H5 dataset
 - [ ] Read from raw bags (to produce aligned/unaligned H5 datasets)
 - [ ] Polar to Cartesian Projection
 - [ ] Camera/Radar coordinate transforms

## Install

`pip install radical-dataset`

## How to use